# Fake News Detection

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

## Importing Dataset

In [1]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
df_true["text"] = df_true["text"].replace("(Reuters)","",regex=True)
df_true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON () - The head of a conservative Rep...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON () - Transgender people will be all...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON () - The special counsel investigat...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON () - Trump campaign adviser George ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON () - President Donald Trump...,politicsNews,"December 29, 2017"


## Inserting a column "class" as target feature

In [ ]:
df_fake["class"] = 0
df_true["class"] = 1

In [ ]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [ ]:
# Removing last 10 rows for manual testing
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
    df_fake.drop([i], axis = 0, inplace = True)


df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
    df_true.drop([i], axis = 0, inplace = True)

In [ ]:
df_fake.shape, df_true.shape

((23471, 5), (21407, 5))

In [ ]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

<ipython-input-13-3aaf8ec2aad1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"] = 0
<ipython-input-13-3aaf8ec2aad1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["class"] = 1


In [ ]:
df_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [ ]:
df_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO () - Cesar Mata Pires, the owner and...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA () - North Korea and the United States ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA () - North Korea and the United States ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN () - Danish police said on Tuesday ...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS () - Two North Korean shipments...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS () - NATO allies on Tuesday welcomed ...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON () - LexisNexis, a provider of legal, r...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK () - In the shadow of disused Soviet-era...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW () - Vatican Secretary of State Cardina...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA () - Indonesia will buy 11 Sukhoi figh...,worldnews,"August 22, 2017",1


In [ ]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

## Merging True and Fake Dataframes

In [ ]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [ ]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

## Removing columns which are not required

In [ ]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [ ]:
df.isnull().sum()

text     0
class    0
dtype: int64

## Random Shuffling the dataframe

In [ ]:
df = df.sample(frac = 1)

In [ ]:
df.head()

,text,class
4442,WASHINGTON () - The U.S. Senate is expected to...,1
3816,WASHINGTON () - A major challenge to the U.S. ...,1
4960,WASHINGTON () - President Donald Trump plans t...,1
6410,WASHINGTON () - The nominee to be U.S. Preside...,1
3877,This election has been hard for South Carolina...,0


In [ ]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [ ]:
df.columns

Index(['text', 'class'], dtype='object')

In [ ]:
df.head()

,text,class
0,WASHINGTON () - The U.S. Senate is expected to...,1
1,WASHINGTON () - A major challenge to the U.S. ...,1
2,WASHINGTON () - President Donald Trump plans t...,1
3,WASHINGTON () - The nominee to be U.S. Preside...,1
4,This election has been hard for South Carolina...,0


## Creating a function to process the texts

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]','',text)
    # This is where i remove the "()" from the text column. You can do in whatever way you want
    # The key is to remove the "(Reuters)" string as it is present in all text of True.csv.
    # The Model during the training part can memorize it and perfrom great in training and badly when other testing input is given.
    text = re.sub('[()]','',text)
    text = re.sub('\\W',' ',text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
df["text"] = df["text"].apply(wordopt)

## Defining dependent and independent variables

In [ ]:
x = df["text"]
y = df["class"]

## Splitting Training and Testing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

## Convert text to vectors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [ ]:
pred_lr=LR.predict(xv_test)

In [ ]:
LR.score(xv_test, y_test)

0.979590017825312

In [ ]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      5903
           1       0.98      0.98      0.98      5317

    accuracy                           0.98     11220
   macro avg       0.98      0.98      0.98     11220
weighted avg       0.98      0.98      0.98     11220



## Decision Tree Classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

0.9538324420677362

In [ ]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      5903
           1       0.96      0.94      0.95      5317

    accuracy                           0.95     11220
   macro avg       0.95      0.95      0.95     11220
weighted avg       0.95      0.95      0.95     11220



## Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [ ]:
pred_gbc = GBC.predict(xv_test)

In [ ]:
GBC.score(xv_test, y_test)

0.9665775401069518

In [ ]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       0.97      0.96      0.97      5903
           1       0.96      0.97      0.97      5317

    accuracy                           0.97     11220
   macro avg       0.97      0.97      0.97     11220
weighted avg       0.97      0.97      0.97     11220



## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
pred_rfc = RFC.predict(xv_test)

In [ ]:
RFC.score(xv_test, y_test)

0.9779857397504457

In [ ]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      5903
           1       0.98      0.98      0.98      5317

    accuracy                           0.98     11220
   macro avg       0.98      0.98      0.98     11220
weighted avg       0.98      0.98      0.98     11220



## Model Testing

In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)


    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),                                                                                                       output_lable(pred_DT[0]),
                                                                                                              output_lable(pred_GBC[0]),
                                                                                                              output_lable(pred_RFC[0])))

In [ ]:
news = str(input())
manual_testing(news)

Russia has launched its biggest drone attack on Kyiv since its full-scale invasion of Ukraine began last year, the city's mayor has said.  Residents were woken by explosions before dawn on Saturday, and for more than six hours, the booms of Kyiv's air defences echoed through the city.  There was wave after wave of attacks from the north and east.  Officials said more than 75 Iranian-made Shahed drones were fired at the capital - all but one were shot down.  With Russia's dwindling missile stocks, Shahed drones are seen as a cheap alternative. They are slower than ballistic missiles and have a distinctive wingspan.  It was a night where the whines of their engines blended with the booms of the city's air defences.  As ever, even if a missile or drone is intercepted, the falling debris can be lethal too.  There have been no reported deaths from this attack, but at least five people were injured, including an 11-year-old child, Kyiv's mayor Vitaliy Klitschko said.  A kindergarten was amon

### **Cross-validation**


## Logistic Regression

In [ ]:
model = LogisticRegression()

# Инициализация StratifiedKFold с 10 фолдами
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Переменные для хранения результатов кросс-валидации
results = []

# Инициализация TF-IDF векторизатора
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Проходим по каждому фолду
for fold, (train_index, test_index) in enumerate(kf.split(x, y), 1):
    x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # Преобразуем текст в числовой формат с использованием TF-IDF
    x_train_tfidf = tfidf_vectorizer.fit_transform(x_train_fold)
    x_test_tfidf = tfidf_vectorizer.transform(x_test_fold)

    # Обучаем модель на тренировочных данных
    model.fit(x_train_tfidf, y_train_fold)

    # Получаем предсказания на тестовых данных
    y_pred_fold = model.predict(x_test_tfidf)

    # Оцениваем точность модели
    accuracy_fold = accuracy_score(y_test_fold, y_pred_fold)

    # Сохраняем результаты в виде словаря
    result_dict = {
        'Fold': fold,
        'Accuracy': accuracy_fold
    }

    results.append(result_dict)

# Создаем DataFrame из результатов
results_df = pd.DataFrame(results)

# Выводим таблицу результатов
print(results_df)

   Fold  Accuracy
0     1  0.980838
1     2  0.981506
2     3  0.983066
3     4  0.981952
4     5  0.983512
5     6  0.988859
6     7  0.982398
7     8  0.978164
8     9  0.981056
9    10  0.983508


## Gradient Boosting Classifier

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
# Инициализация модели дерева решений
model = DecisionTreeClassifier(random_state=42)

# Инициализация StratifiedKFold с 10 фолдами
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Переменные для хранения результатов кросс-валидации
results_dt = []

# Проходим по каждому фолду
for fold, (train_index, test_index) in enumerate(kf.split(x, y), 1):
    x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # Обучаем модель на тренировочных данных
    model.fit(x_train_fold, y_train_fold)

    # Получаем предсказания на тестовых данных
    y_pred_fold = model.predict(x_test_fold)

    # Оцениваем точность модели
    accuracy_fold = accuracy_score(y_test_fold, y_pred_fold)

    # Сохраняем результаты в виде словаря
    result_dict = {
        'Fold': fold,
        'Accuracy': accuracy_fold
    }

    results_dt.append(result_dict)

# Создаем DataFrame из результатов
results_dt_df = pd.DataFrame(results_dt)

# Выводим таблицу результатов
print(results_dt_df)

NameError: ignored

## Random Forest Classifier


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# x - признаки, y - метки
# Предполагается, что x и y уже определены

# Инициализация модели Random Forest Classifier
model_rf = RandomForestClassifier(random_state=42)

# Инициализация StratifiedKFold с 10 фолдами
kf_rf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Переменные для хранения результатов кросс-валидации
results_rf = []

# Проходим по каждому фолду
for fold, (train_index, test_index) in enumerate(kf_rf.split(x, y), 1):
    x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # Обучаем модель на тренировочных данных
    model_rf.fit(x_train_fold, y_train_fold)

    # Получаем предсказания на тестовых данных
    y_pred_fold_rf = model_rf.predict(x_test_fold)

    # Оцениваем точность модели
    accuracy_fold_rf = accuracy_score(y_test_fold, y_pred_fold_rf)

    # Сохраняем результаты в виде словаря
    result_dict_rf = {
        'Fold': fold,
        'Accuracy': accuracy_fold_rf
    }

    results_rf.append(result_dict_rf)

# Создаем DataFrame из результатов
results_rf_df = pd.DataFrame(results_rf)

# Выводим таблицу результатов
print(results_rf_df)

## Decision Tree Classification

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# x - признаки, y - метки
# Предполагается, что x и y уже определены

# Инициализация модели Decision Tree Classifier
model_dt = DecisionTreeClassifier(random_state=42)

# Инициализация StratifiedKFold с 10 фолдами
kf_dt = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Переменные для хранения результатов кросс-валидации
results_dt = []

# Проходим по каждому фолду
for fold, (train_index, test_index) in enumerate(kf_dt.split(x, y), 1):
    x_train_fold, x_test_fold = x.iloc[train_index], x.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # Обучаем модель на тренировочных данных
    model_dt.fit(x_train_fold, y_train_fold)

    # Получаем предсказания на тестовых данных
    y_pred_fold_dt = model_dt.predict(x_test_fold)

    # Оцениваем точность модели
    accuracy_fold_dt = accuracy_score(y_test_fold, y_pred_fold_dt)

    # Сохраняем результаты в виде словаря
    result_dict_dt = {
        'Fold': fold,
        'Accuracy': accuracy_fold_dt
    }

    results_dt.append(result_dict_dt)

# Создаем DataFrame из результатов
results_dt_df = pd.DataFrame(results_dt)

# Выводим таблицу результатов
print(results_dt_df)